In [45]:
import numpy as np
import librosa
import soundfile as sf
from pydub import AudioSegment
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import os

class IkedaMultiSampleLoop:
    def __init__(self, sample_folder_path, bpm=160):
        """
        Ryoji Ikeda-inspired IDM loop generator with multiple samples
        
        Parameters:
        - sample_folder_path: folder containing your .wav samples
        - bpm: tempo for the loop
        """
        self.sample_folder = sample_folder_path
        self.bpm = bpm
        self.sample_rate = 44100
        self.beat_duration = 60 / bpm  # seconds per beat
        
        # Load all samples from folder
        self.samples = self.load_all_samples(sample_folder_path)
        print(f"Loaded {len(self.samples)} samples")
        
        # Categorize samples by characteristics
        self.categorized_samples = self.categorize_samples()
    
    def load_all_samples(self, folder_path):
        """Load all .wav files from a folder"""
        samples = {}
        
        for filename in os.listdir(folder_path):
            if filename.lower().endswith('.wav'):
                filepath = os.path.join(folder_path, filename)
                try:
                    audio, sr = librosa.load(filepath, sr=self.sample_rate)
                    
                    # Ensure samples are short (trim if needed)
                    if len(audio) > int(self.sample_rate * 0.3):  # 300ms max
                        audio = audio[:int(self.sample_rate * 0.3)]
                    
                    # Normalize
                    if np.max(np.abs(audio)) > 0:
                        audio = audio / np.max(np.abs(audio)) * 0.8
                    
                    samples[filename] = {
                        'audio': audio,
                        'length': len(audio),
                        'duration': len(audio) / sr,
                        'filename': filename,
                        'pitch': self.estimate_pitch(audio),
                        'energy': np.mean(np.abs(audio))
                    }
                    
                except Exception as e:
                    print(f"Error loading {filename}: {e}")
        
        return samples
    
    def estimate_pitch(self, audio):
        """Simple pitch estimation for categorization"""
        try:
            if len(audio) > 2048:
                freqs = np.fft.rfftfreq(len(audio), 1/self.sample_rate)
                fft = np.abs(np.fft.rfft(audio))
                if len(fft) > 0:
                    dominant_freq = freqs[np.argmax(fft[1:]) + 1]
                    return dominant_freq
        except:
            pass
        return 1000  # Default
    
    def categorize_samples(self):
        """Categorize samples by their sonic characteristics"""
        categories = {
            'high_click': [],    # High frequency, short
            'low_thud': [],      # Low frequency
            'mid_punch': [],     # Mid-range
            'noise': [],         # Noise-based
            'tonal': [],         # Tonally rich
            'metallic': [],      # Metallic character
            'glitch': []         # Digital artifacts
        }
        
        for name, sample in self.samples.items():
            pitch = sample['pitch']
            energy = sample['energy']
            duration = sample['duration']
            
            # Categorization logic
            if pitch > 3000 and duration < 0.05:
                categories['high_click'].append(name)
            elif pitch < 200:
                categories['low_thud'].append(name)
            elif 'noise' in name.lower() or 'hiss' in name.lower():
                categories['noise'].append(name)
            elif 'metal' in name.lower() or 'ting' in name.lower():
                categories['metallic'].append(name)
            elif pitch > 1000 and pitch < 3000:
                categories['mid_punch'].append(name)
            elif 'glitch' in name.lower() or 'error' in name.lower():
                categories['glitch'].append(name)
            else:
                categories['tonal'].append(name)
        
        # Fill empty categories
        all_samples = list(self.samples.keys())
        for cat, samples in categories.items():
            if len(samples) == 0:
                categories[cat] = all_samples.copy()
        
        return categories
    
    def get_sample_by_category(self, category, pattern_position):
        """Get a sample from a specific category with some variation logic"""
        available = self.categorized_samples.get(category, [])
        if not available:
            available = list(self.samples.keys())
        
        # Weight recent samples less to avoid repetition
        if hasattr(self, 'recent_samples'):
            weights = [0.5 if s in self.recent_samples[-3:] else 1.0 
                      for s in available]
            # Normalize weights
            weights = np.array(weights) / np.sum(weights)
            selected = np.random.choice(available, p=weights)
        else:
            selected = random.choice(available)
        
        # Track recent samples
        if not hasattr(self, 'recent_samples'):
            self.recent_samples = []
        self.recent_samples.append(selected)
        if len(self.recent_samples) > 10:
            self.recent_samples.pop(0)
        
        return selected
    
    def create_idm_pattern(self, bars=4, subdivision=16):
        """
        Create an IDM-style rhythmic pattern with sample assignments
        Returns pattern with (velocity, category) tuples
        """
        pattern_length = bars * subdivision
        pattern = []
        
        for i in range(pattern_length):
            bar_pos = i % 16
            beat_in_bar = bar_pos // 4
            
            # Determine sample category based on position
            if bar_pos == 0:  # Downbeat
                category = random.choice(['low_thud', 'mid_punch'])
                velocity = 1.0
            elif bar_pos in [4, 8, 12]:  # Other strong beats
                category = random.choice(['mid_punch', 'tonal'])
                velocity = 0.9
            elif bar_pos % 2 == 1:  # Offbeats
                category = random.choice(['high_click', 'metallic'])
                velocity = 0.7
            elif bar_pos in [2, 6, 10, 14]:  # Upbeats
                category = random.choice(['high_click', 'glitch'])
                velocity = 0.8
            else:
                # Fill positions
                if random.random() > 0.6:
                    category = random.choice(list(self.categorized_samples.keys()))
                    velocity = random.uniform(0.3, 0.6)
                else:
                    category = None
                    velocity = 0
            
            # Add occasional bursts
            if random.random() > 0.95:  # 5% chance of fill
                fill_length = random.randint(3, 6)
                for j in range(min(fill_length, pattern_length - i)):
                    pattern.append(('glitch', random.uniform(0.4, 0.8)))
                i += fill_length - 1
                continue
            
            pattern.append((category, velocity))
        
        return pattern
    
    def apply_sample_processing(self, sample_audio, pattern_position, velocity):
        """Apply processing to individual samples"""
        processed = sample_audio.copy()
        
        # 1. Velocity-based gain
        processed = processed * velocity
        
        # 2. Pitch shifting based on position
        if random.random() > 0.7:
            pitch_shift = random.choice([-12, -7, 0, 5, 7, 12])
            if pitch_shift != 0:
                try:
                    processed = librosa.effects.pitch_shift(
                        processed, 
                        sr=self.sample_rate, 
                        n_steps=pitch_shift
                    )
                except:
                    pass
        
        # 3. Reverse sometimes
        if random.random() > 0.9:
            processed = processed[::-1]
        
        # 4. Bitcrush occasionally
        if random.random() > 0.8:
            bits = random.choice([4, 8, 12])
            processed = np.round(processed * (2**bits - 1)) / (2**bits - 1)
        
        # 5. Time stretch (micro)
        if random.random() > 0.95:
            stretch = random.uniform(0.5, 1.5)
            if len(processed) > 100:
                try:
                    processed = librosa.effects.time_stretch(processed, rate=stretch)
                except:
                    pass
        
        # 6. Add digital noise
        if random.random() > 0.9:
            noise = np.random.normal(0, 0.02 * velocity, len(processed))
            processed = processed + noise
        
        return processed
    
    def create_multi_sample_loop(self, bars=4, output_path="ikeda_multisample_loop.wav"):
        """
        Generate a 4-bar loop using multiple samples
        """
        subdivision = 16  # 16th notes
        pattern = self.create_idm_pattern(bars, subdivision)
        
        # Reset recent samples tracking
        self.recent_samples = []
        
        # Calculate timing
        sixteenth_duration = self.beat_duration / 4
        total_duration = bars * 4 * self.beat_duration
        total_samples = int(total_duration * self.sample_rate)
        
        # Create empty audio buffer
        loop_audio = np.zeros(total_samples)
        
        # Track sample usage for reporting
        sample_usage = defaultdict(int)
        
        # Place samples according to pattern
        for i, (category, velocity) in enumerate(pattern):
            if velocity > 0 and category:
                # Get appropriate sample
                sample_name = self.get_sample_by_category(category, i)
                sample_usage[sample_name] += 1
                
                sample_data = self.samples[sample_name]
                base_audio = sample_data['audio']
                
                # Apply processing
                processed_click = self.apply_sample_processing(base_audio, i, velocity)
                
                # Calculate start position
                start_time = i * sixteenth_duration
                start_sample = int(start_time * self.sample_rate)
                
                # Add timing humanization (micro-variations)
                timing_offset = random.randint(-5, 5)
                actual_start = max(0, start_sample + timing_offset)
                
                # Add to loop
                end_pos = min(len(loop_audio), actual_start + len(processed_click))
                click_len = end_pos - actual_start
                
                if click_len > 0:
                    loop_audio[actual_start:actual_start + click_len] += processed_click[:click_len]
        
        # Normalize to prevent clipping
        peak = np.max(np.abs(loop_audio))
        if peak > 0:
            loop_audio = loop_audio / peak * 0.8
        
        # Save the loop
        sf.write(output_path, loop_audio, self.sample_rate)
        
        # Print sample usage statistics
        print(f"\n=== Loop Generation Complete ===")
        print(f"BPM: {self.bpm}, Bars: {bars}")
        print(f"Total samples used: {sum(sample_usage.values())}")
        print(f"Unique samples used: {len(sample_usage)}")
        print(f"\nSample usage breakdown:")
        for sample, count in sorted(sample_usage.items(), key=lambda x: x[1], reverse=True)[:10]:
            print(f"  {sample}: {count} hits")
        
        return loop_audio, pattern, sample_usage
    
    def create_layered_loop(self, bars=4, layers=3, output_path="layered_loop.wav"):
        """Create a multi-layered loop with different sample sets per layer"""
        all_layers = []
        
        for layer in range(layers):
            print(f"\nGenerating layer {layer + 1}/{layers}")
            
            # Different parameters per layer
            if layer == 0:  # Base layer
                # Focus on low frequencies
                self.categorized_samples = {k: v for k, v in self.categorized_samples.items() 
                                          if k in ['low_thud', 'mid_punch']}
                layer_audio, _, _ = self.create_multi_sample_loop(bars, f"layer_{layer}.wav")
                
            elif layer == 1:  # Mid layer
                # Focus on mid/high frequencies
                self.categorized_samples = {k: v for k, v in self.categorized_samples.items() 
                                          if k in ['high_click', 'metallic', 'tonal']}
                layer_audio, _, _ = self.create_multi_sample_loop(bars, f"layer_{layer}.wav")
                
            else:  # Top layer (glitch/effects)
                # Only glitch and noise
                self.categorized_samples = {k: v for k, v in self.categorized_samples.items() 
                                          if k in ['glitch', 'noise']}
                layer_audio, _, _ = self.create_multi_sample_loop(bars, f"layer_{layer}.wav")
            
            all_layers.append(layer_audio)
        
        # Mix layers with different volumes
        mixed = np.zeros_like(all_layers[0])
        layer_volumes = [0.8, 0.6, 0.4]  # Base louder, top layers quieter
        
        for i, layer in enumerate(all_layers):
            if len(layer) > len(mixed):
                layer = layer[:len(mixed)]
            mixed[:len(layer)] += layer[:len(mixed)] * layer_volumes[i]
        
        # Normalize
        peak = np.max(np.abs(mixed))
        if peak > 0:
            mixed = mixed / peak * 0.8
        
        sf.write(output_path, mixed, self.sample_rate)
        print(f"\nLayered loop saved to: {output_path}")
        
        return mixed
    
    def export_sample_sheet(self, pattern, sample_usage, output_file="pattern_report.txt"):
        """Export a text report of the pattern and sample usage"""
        with open(output_file, 'w') as f:
            f.write(f"Ryoji Ikeda-style IDM Loop Pattern\n")
            f.write(f"BPM: {self.bpm}\n")
            f.write(f"Time Signature: 4/4\n")
            f.write(f"Bars: {len(pattern)//16}\n\n")
            
            f.write("PATTERN GRID:\n")
            f.write("Pos | Cat       | Vel | Sample\n")
            f.write("-" * 50 + "\n")
            
            for i, (category, velocity) in enumerate(pattern):
                if i % 16 == 0:
                    f.write(f"\nBAR {i//16 + 1}:\n")
                
                if velocity > 0:
                    # Find which sample would be used here
                    sample_name = self.get_sample_by_category(category, i) if category else "None"
                    f.write(f"{i:3d} | {category or 'None':10} | {velocity:.2f} | {sample_name[:20]}\n")
            
            f.write("\n\nSAMPLE USAGE STATISTICS:\n")
            f.write("-" * 50 + "\n")
            for sample, count in sorted(sample_usage.items(), key=lambda x: x[1], reverse=True):
                f.write(f"{sample}: {count} hits\n")
   
   
   
   


In [46]:
# Usage example

bpm = 170
output_folder = 'test1'

# Usage example
if __name__ == "__main__":
    # Initialize with folder containing your samples
    generator = IkedaMultiSampleLoop(
        sample_folder_path="sound",
        bpm=bpm
    )
    
    # Create a basic multi-sample loop
    loop_audio, pattern, sample_usage = generator.create_multi_sample_loop(
        bars=4,
        output_path=f"{output_folder}/ikeda_idm_loop_{bpm}_bpm_multi_sample_loop.wav"
    )
    
    # Export pattern report
    generator.export_sample_sheet(pattern, sample_usage,f"{output_folder}/ikeda_idm_loop_{bpm}_bpm_pattern_report.txt")
    
    # Create a complex layered loop
    layered_audio = generator.create_layered_loop(
        bars=4,
        layers=3,
        output_path=f"{output_folder}/ikeda_idm_loop_{bpm}_bpm_complex_layered_loop.wav"
    ) 


Loaded 13 samples

=== Loop Generation Complete ===
BPM: 170, Bars: 4
Total samples used: 74
Unique samples used: 13

Sample usage breakdown:
  d3.wav: 11 hits
  jalastram_snare_05.wav: 9 hits
  test_kick_74.wav: 8 hits
  test_kick_64.wav: 7 hits
  test_kick_33.wav: 7 hits
  test_kick_100.wav: 7 hits
  test_kick_18.wav: 5 hits
  pulse.wav: 5 hits
  test_kick_53.wav: 4 hits
  test_kick_3.wav: 4 hits

Generating layer 1/3

=== Loop Generation Complete ===
BPM: 170, Bars: 4
Total samples used: 77
Unique samples used: 13

Sample usage breakdown:
  d3.wav: 11 hits
  test_kick_3.wav: 8 hits
  test_kick_74.wav: 7 hits
  Kick_Low_Tone_One_Shot.wav: 7 hits
  test_kick_125.wav: 6 hits
  test_kick_33.wav: 6 hits
  shhhhhh1.wav: 6 hits
  jalastram_snare_05.wav: 5 hits
  test_kick_18.wav: 5 hits
  pulse.wav: 5 hits

Generating layer 2/3

=== Loop Generation Complete ===
BPM: 170, Bars: 4
Total samples used: 79
Unique samples used: 13

Sample usage breakdown:
  d3.wav: 8 hits
  test_kick_125.wav: 8 

In [37]:
def create_granular_loop(sample_path, grain_size=500, overlap=0.99):
    """
    Create granular texture from sample
    """
    audio, sr = librosa.load(sample_path, sr=None)
    
    grain_samples = int(sr * grain_size / 1000)  # grain_size in ms
    hop_samples = int(grain_samples * (1 - overlap))
    
    grains = []
    for start in range(0, len(audio) - grain_samples, hop_samples):
        grain = audio[start:start + grain_samples]
        
        # Apply window
        window = np.hanning(len(grain))
        grain = grain * window
        
        # Random pitch shift occasionally
        if random.random() > 0.5:
            grain = librosa.effects.pitch_shift(grain, sr=sr, n_steps=random.uniform(-24, 24))
        
        grains.append(grain)
    
    # Rearrange grains randomly
    random.shuffle(grains)
    
    # Combine grains
    output = np.concatenate(grains[:min(50, len(grains))])
    
    sf.write("granular_texture.wav", output, sr)
    return output



create_granular_loop(f"idm_loop_{bpm}bpm.wav")

array([-2.24960321e-17,  3.41752350e-17,  2.51738126e-17, ...,
        3.34534212e-13, -7.33910112e-13, -4.81372971e-12])